<a href="https://colab.research.google.com/github/Prach1/CS69002_9A_18CS60R30/blob/master/DL_Assign9a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import pandas as pd
import numpy as np
import io
import re

In [0]:
from google.colab import files
uploaded = files.upload()

In [148]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [149]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/Train_20K.csv', sep='\t')
print(df.head())

                                                text  label
0  John Waters has given us a genuinely enjoyable...      1
1  This first two seasons of this comedy series w...      1
2  What an unfortunate mess is "Shiner." I wanted...      0
3  I'm not entirely sure Rob Schmidt qualifies as...      1
4  i wasn't sure whether to laugh or cry. Porrett...      0


In [150]:
df1 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data/Test_5K.csv', sep='\t')
print(df1.head())

                                                text  label
0  as always this is an inaccurate picture of the...      0
1  Did the movie-makers even preview this before ...      0
2  Heavily re-edited and often confusing, the ori...      0
3  I notice that most of the people who think thi...      0
4  First of all, this is a low-budget movie, so m...      0


In [0]:
for chunk in df:
  print(chunk)

text
label


In [6]:
# for i, j in df.iterrows(): 
#     print(i)
#     print(j)

print(type(df['text']))
print(type(df['label']))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [151]:
text_reviews = df['text'].astype(str).tolist()
text_labels = df['label'].astype(int)
print(type(text_labels))

text_reviews = [x.lower() for x in text_reviews]
#print(type(text_reviews))
#print(text_reviews[0])

<class 'pandas.core.series.Series'>


In [0]:
text_test_reviews = df1['text'].astype(str).tolist()
text_test_labels = df1['label'].astype(int)
#print(type(text_labels))

text_test_reviews = [x.lower() for x in text_test_reviews]
#print(type(text_reviews))
#print(text_reviews[0])

In [0]:
import re

def preprocess(text):    
  text = re.sub(r'<br.*?>',' ',text)
  text = re.sub(r'[^\w\s]',' ',text) 
  text = re.sub(r'[0-9]+', ' ', text)
  
  return text
  

In [154]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer

train_data = []
train_data_X = []
stop = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

for review in text_reviews:
  new_review = preprocess(review)
  train_data.append(new_review)
  
print(train_data[0])

for review in train_data:
  review = [lemmatizer.lemmatize(word) for word in review.split() if word not in stop and word != '' and len(lemmatizer.lemmatize(word)) > 1]
  train_data_X.append(review)
  
print(train_data_X[0])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
john waters has given us a genuinely enjoyable film  this certainly isn t without its shocking waters esque moments  but it is tamer than his older culty stuff  such as  pink flamingoes    pecker  harkens back to john s early mainstream stage in that it reminds the viewer of the same kind of humor that was evident in  polyester   overall  a really fun comedy with some great moments 
['john', 'water', 'given', 'genuinely', 'enjoyable', 'film', 'certainly', 'without', 'shocking', 'water', 'esque', 'moment', 'tamer', 'older', 'culty', 'stuff', 'pink', 'flamingo', 'pecker', 'harkens', 'back', 'john', 'early', 'mainstream', 'stage', 'reminds', 'viewer', 'kind', 'humor', 'evident', 'polyester', 'overall', 'really', 'fun', 'comedy', 'great', 'moment']


In [155]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
#nltk.download('stopwords')
#nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer

test_data = []
test_data_X = []
stop = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

for review in text_test_reviews:
  new_review = preprocess(review)
  test_data.append(new_review)
  
print(test_data[0])

for review in test_data:
  review = [lemmatizer.lemmatize(word) for word in review.split() if word not in stop and word != '' and len(lemmatizer.lemmatize(word)) > 1]
  test_data_X.append(review)
  
print(test_data_X[0])

as always this is an inaccurate picture of the homeless  tv told a lot of lies about panhandlers in the early  s and made everyone look bad  and claimed we all made over    a day when      a day was much closer to reality  when someone drove by where i held up a sign offering to work  and offered me work  i actually went and took the work if i was physically able and if i would been offered the      id damned sure invested in in apt prepaid for at least   years  and kept most in the bank and still left myself      for nl      and      cash games at the casinos  i usually always win and could win decent if i just had a bankroll  instead i win about    a month is all playing in always minimum buying in due to not wanting to risk losing it all  i was only homeless cause i didn t wanna risk spending all my money and going broke  sometimes i had over      in my sock while i slept outside  anyone wanting to talk contact sevencard  on yahoo messenger i admit i was different than most homeless

In [0]:
def generate_word_frequency(dataset):
  word_to_frequency = {}
 
  for sent in dataset:
    for word in sent:
      if word not in word_to_frequency:
        word_to_frequency[word] = 1
      else:
        word_to_frequency[word] += 1
              
  return word_to_frequency

In [0]:
def generate_bigram_word_frequency(dataset):
  word_to_frequency = {}
  
  for sent in dataset:
    for index,word1 in enumerate(sent[:-1]):
      word2 = sent[index+1]
      if (word1,word2) not in word_to_frequency:
        word_to_frequency[(word1,word2)] = 1
      else:
        word_to_frequency[(word1,word2)] += 1
              
  return word_to_frequency  

In [0]:
def generate_word_ids(dataset):
  word_to_ix = {}
 
  for word in dataset:
    if word not in word_to_ix:
      word_to_ix[word] = len(word_to_ix)
              
  word_to_ix['UNKNOWN'] = len(word_to_ix) 
              
  return word_to_ix


In [0]:
def generate_bigram_word_ids(dataset):
  word_to_ix = {}
 
  for word in dataset:
    if word not in word_to_ix:
      word_to_ix[word] = len(word_to_ix)
              
  word_to_ix['UNKNOWN'] = len(word_to_ix) 
              
  return word_to_ix

In [158]:
word_to_frequency = generate_word_frequency(train_data_X + test_data_X)
# for key in word_to_bigram_frequency:
#   print(key)
#   break

new_word_dataset = []

for word in word_to_frequency:
  if(word_to_frequency[word] >= 3):
    new_word_dataset.append(word)
        
print(len(word_to_frequency))
print(len(new_word_dataset))

word_to_ix = generate_word_ids(new_word_dataset)

print(len(word_to_ix))
print(word_to_ix['UNKNOWN'])

VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2

63726
32047
32048
32047


In [133]:
word_to_bigram_frequency = generate_bigram_word_frequency(train_data_X + test_data_X)
new_bigram_word_dataset = []

for word in word_to_bigram_frequency:
  if(word_to_bigram_frequency[word] >= 7):
    new_bigram_word_dataset.append(word)
    
print(len(word_to_bigram_frequency))
print(len(new_bigram_word_dataset))
#print(new_bigram_word_dataset[0])

word_to_bigram_ix = generate_bigram_word_ids(new_bigram_word_dataset)
BIGRAM_VOCAB_SIZE = len(word_to_bigram_ix)
print(BIGRAM_VOCAB_SIZE)

1554114
39242
39243


In [0]:
def make_bow_vector(sentence):
    # create a vector of zeros of vocab size = len(word_to_idx)
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        if word not in word_to_ix:
            #raise ValueError('Word',word,' not present in the dictionary. Sorry!')
            vec[word_to_ix['UNKNOWN']]+=1
        else:
            vec[word_to_ix[word]]+=1
    return vec.view(1, -1)

# def make_target(label, label_to_ix):
#     return torch.LongTensor([label_to_ix[label]])
    #return torch.LongTensor(label)

In [0]:
def make_bigram_bow_vector(sentence):
    # create a vector of zeros of vocab size = len(word_to_idx)
    vec = torch.zeros(len(word_to_bigram_ix))
    for index,word1 in enumerate(sentence[:-1]):
      word2 = sentence[index+1]
      if (word1,word2) not in word_to_bigram_ix:
        vec[word_to_bigram_ix['UNKNOWN']]+=1
      else:
        vec[word_to_bigram_ix[(word1,word2)]]+=1
    return vec.view(1, -1)

In [0]:
train = []

for review in train_data_X:
  embedding = torch.autograd.Variable(make_bow_vector(review)).cuda()
  train.append(embedding)

In [0]:
bigram_train = []

for review in train_data_X:
  embedding = torch.autograd.Variable(make_bigram_bow_vector(review)).cuda()
  bigram_train.append(embedding)

In [0]:
test = []
for review in test_data_X:
  embedding = torch.autograd.Variable(make_bow_vector(review)).cuda()
  test.append(embedding)

In [0]:
bigram_test = []
for review in test_data_X:
  embedding = torch.autograd.Variable(make_bigram_bow_vector(review)).cuda()
  bigram_test.append(embedding)

In [0]:
print(train[0])
print(len(train))
print(test[4])
print(len(test))

tensor([[2., 2., 1.,  ..., 0., 0., 1.]], device='cuda:0')
17999
tensor([[0., 0., 0.,  ..., 0., 0., 1.]], device='cuda:0')
5000


In [0]:
# data = []

# for index,review in enumerate(train):
#   data.append((review,text_labels[index]))
  
# print(data[0])

#label_to_tensor = torch.LongTensor(text_labels)

label_to_tensor = []

#print(text_labels)
text_labels1 = []

for label in text_labels:
  text_labels1.append([label])
  

#print(text_labels1)

for label in text_labels1:
  label_to_tensor.append(torch.tensor(label, dtype = torch.long))
  #print(label_to_tensor)
  
#print(label_to_tensor)

#for label in label_to_tensor:
  

In [0]:
label_to_test_tensor = []

text_test_labels1 = []

for label in text_test_labels:
  text_test_labels1.append([label])

for label in text_test_labels1:
  label_to_test_tensor.append(torch.tensor(label, dtype = torch.long))
  
#print(label_to_test_tensor)
  

In [164]:
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

SEED = 42

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.device('cuda') 

device(type='cuda')

In [0]:
class Task1(nn.Module):
  def __init__(self, num_labels,hidden,vocab_size):
    super(Task1, self).__init__()
    self.lin1 = nn.Linear(vocab_size, hidden)
    self.lin2 = nn.Linear(hidden, NUM_LABELS)
    
  def forward(self, x):
    out = self.lin1(x)
    out = F.softmax(self.lin2(out))
    
    return out

In [0]:
class Task2(nn.Module):
  def __init__(self, num_labels,hidden1,hidden2,vocab_size):
    super(Task2, self).__init__()
    self.lin1 = nn.Linear(vocab_size, hidden1)
    self.lin2 = nn.Linear(hidden1, hidden2)
    self.lin3 = nn.Linear(hidden2, NUM_LABELS)
    
  def forward(self, x):
    out = self.lin1(x)
    out = F.relu(self.lin2(out))
    out = F.softmax(self.lin3(out))
    
    return out

In [0]:
class Task3(nn.Module):
  def __init__(self, num_labels,hidden1,hidden2,hidden3,vocab_size):
    super(Task3, self).__init__()
    self.lin1 = nn.Linear(vocab_size, hidden1)
    self.lin2 = nn.Linear(hidden1, hidden2)
    self.lin3 = nn.Linear(hidden2, hidden3)
    self.lin4 = nn.Linear(hidden3, NUM_LABELS)
    
  def forward(self, x):
    out = self.lin1(x)
    out = F.relu(self.lin2(out))
    out = self.lin3(out)
    out = F.softmax(self.lin4(out))
    
    return out

In [0]:
class Task4(nn.Module):
  def __init__(self, num_labels,hidden1,hidden2,hidden3,vocab_size):
    super(Task4, self).__init__()
    self.lin1 = nn.Linear(vocab_size, hidden1)
    torch.nn.Dropout(0.5)
    self.lin2 = nn.Linear(hidden1, hidden2)
    torch.nn.Dropout(0.5)
    self.lin3 = nn.Linear(hidden2, hidden3)
    torch.nn.Dropout(0.5)
    self.lin4 = nn.Linear(hidden3, NUM_LABELS)
    
  def forward(self, x):
    out = self.lin1(x)
    out = F.relu(self.lin2(out))
    out = self.lin3(out)
    out = F.softmax(self.lin4(out))
    
    return out

In [0]:
class Task6(nn.Module):
  def __init__(self, num_labels,hidden1,hidden2,hidden3,vocab_size):
    super(Task6, self).__init__()
    self.lin1 = nn.Linear(vocab_size, hidden1)
    torch.nn.Dropout(0.5)
    self.lin2 = nn.Linear(hidden1, hidden2)
    torch.nn.Dropout(0.5)
    self.lin3 = nn.Linear(hidden2, hidden3)
    torch.nn.Dropout(0.5)
    self.lin4 = nn.Linear(hidden3, NUM_LABELS)
    
  def forward(self, x):
    out = self.lin1(x)
    out = F.relu(self.lin2(out))
    out = self.lin3(out)
    out = F.softmax(self.lin4(out))
    
    return out

In [166]:
hidden = 200
bow = Task1(NUM_LABELS,hidden,VOCAB_SIZE).cuda()
print(bow)

Task1(
  (lin1): Linear(in_features=32048, out_features=200, bias=True)
  (lin2): Linear(in_features=200, out_features=2, bias=True)
)


In [22]:
hidden1 = 10
hidden2 = 10
bow = Task2(NUM_LABELS,hidden1,hidden2,VOCAB_SIZE).cuda()
print(bow)

Task2(
  (lin1): Linear(in_features=32048, out_features=10, bias=True)
  (lin2): Linear(in_features=10, out_features=10, bias=True)
  (lin3): Linear(in_features=10, out_features=2, bias=True)
)


In [0]:
hidden1 = 100
hidden2 = 50
hidden3 = 10
bow = Task3(NUM_LABELS,hidden1,hidden2,hidden3,VOCAB_SIZE).cuda()
print(bow)

In [0]:
hidden1 = 30
hidden2 = 20
hidden3 = 10
bow = Task4(NUM_LABELS,hidden1,hidden2,hidden3,VOCAB_SIZE).cuda()
print(bow)

In [140]:
hidden1 = 30
hidden2 = 20
hidden3 = 10
bow = Task6(NUM_LABELS,hidden1,hidden2,hidden3,BIGRAM_VOCAB_SIZE).cuda()
print(bow)

Task6(
  (lin1): Linear(in_features=39243, out_features=30, bias=True)
  (lin2): Linear(in_features=30, out_features=20, bias=True)
  (lin3): Linear(in_features=20, out_features=10, bias=True)
  (lin4): Linear(in_features=10, out_features=2, bias=True)
)


In [0]:
loss_function = nn.NLLLoss()
opt = torch.optim.SGD(bow.parameters(), lr = 0.1)

In [168]:
import time

# the training loop
for epoch in range(10):
  start = time.time()
  for index, instance in enumerate(train):
    bow.zero_grad()
    label = Variable(label_to_tensor[index]).cuda()       
    probs = bow(instance).cuda() # forward pass
    loss = loss_function(probs, label)
    loss.backward()
    opt.step()
  end = time.time()  
  print('epoch is: ' +str(epoch) + ' and loss is: ' +str(loss.data) + ' and time taken is: ' +str(end-start))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


epoch is: 0 and loss is: tensor(-1., device='cuda:0') and time taken is: 37.2442946434021
epoch is: 1 and loss is: tensor(-1.0000, device='cuda:0') and time taken is: 36.94411373138428
epoch is: 2 and loss is: tensor(-1., device='cuda:0') and time taken is: 36.95587968826294
epoch is: 3 and loss is: tensor(-0.9916, device='cuda:0') and time taken is: 37.310444593429565
epoch is: 4 and loss is: tensor(-1., device='cuda:0') and time taken is: 37.168681144714355
epoch is: 5 and loss is: tensor(-0.9998, device='cuda:0') and time taken is: 37.18535256385803
epoch is: 6 and loss is: tensor(-1., device='cuda:0') and time taken is: 37.006630659103394
epoch is: 7 and loss is: tensor(-1., device='cuda:0') and time taken is: 36.97114324569702
epoch is: 8 and loss is: tensor(-1., device='cuda:0') and time taken is: 36.99241662025452
epoch is: 9 and loss is: tensor(-1., device='cuda:0') and time taken is: 36.99219298362732


In [142]:
#Bigram#

import time

# the training loop
for epoch in range(10):
  start = time.time()
  for index, instance in enumerate(bigram_train):
    bow.zero_grad()
    label = Variable(label_to_tensor[index]).cuda()       
    probs = bow(instance).cuda() # forward pass
    loss = loss_function(probs, label)
    loss.backward()
    opt.step()
  end = time.time()  
  print('epoch is: ' +str(epoch) + ' and loss is: ' +str(loss.data) + ' and time taken is: ' +str(end-start))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


epoch is: 0 and loss is: tensor(-5.7215e-14, device='cuda:0') and time taken is: 24.56579089164734
epoch is: 1 and loss is: tensor(-5.7111e-14, device='cuda:0') and time taken is: 24.55873394012451
epoch is: 2 and loss is: tensor(-8.3977e-14, device='cuda:0') and time taken is: 24.469730854034424
epoch is: 3 and loss is: tensor(-1.4182e-13, device='cuda:0') and time taken is: 24.326232194900513
epoch is: 4 and loss is: tensor(-2.6186e-13, device='cuda:0') and time taken is: 24.398052215576172
epoch is: 5 and loss is: tensor(-7.2919e-13, device='cuda:0') and time taken is: 24.407428979873657
epoch is: 6 and loss is: tensor(-5.0875e-14, device='cuda:0') and time taken is: 24.47694230079651
epoch is: 7 and loss is: tensor(-2.9990e-16, device='cuda:0') and time taken is: 24.403366804122925
epoch is: 8 and loss is: tensor(-5.9114e-15, device='cuda:0') and time taken is: 24.32275629043579
epoch is: 9 and loss is: tensor(-0.0271, device='cuda:0') and time taken is: 24.79916000366211


In [171]:
torch.save(bow,'Task1C.mdl')

from google.colab import files
files.download('Task1C.mdl')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task1. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


KeyboardInterrupt: ignored

In [172]:
from google.colab import files
temp_test = files.upload()

Saving Task1C.mdl to Task1C (2).mdl


In [173]:
model = torch.load(io.BytesIO(temp_test['Task1C.mdl']))
print(model)

Task1(
  (lin1): Linear(in_features=32048, out_features=200, bias=True)
  (lin2): Linear(in_features=200, out_features=2, bias=True)
)


In [174]:
print('--- AFTER TRAINING ---')
count = 0
classify = 0
tp = fp = tn = fn = 0
for index, instance in enumerate(test):
    logprobs = model(instance)
    #print(logprobs)
    pred = np.argmax(logprobs.cpu().data.numpy())
    actual = label_to_test_tensor[index][0]
    #print('prediction: {}'.format(pred))
    #print('actual: {}'.format(actual))
    
    if(pred == 1 and actual == 1):
      tp += 1
    elif(pred == 1 and actual == 0):
      fp += 1
    elif(pred == 0 and actual == 1):
      fn += 1
    elif(pred == 0 and actual == 0):
      tn += 1

precision = float(tp / (tp + fp))
recall = float(tp / (tp + fn))
f_score = float((2 * precision * recall) / (precision + recall))
accuracy = float((tp + tn) / (tp + tn + fp + fn))

print(' precision is: ' +str(precision)+' recall is: ' +str(recall)+' F-score is: ' +str(f_score)+ ' accuracy is: '+str(accuracy))
      

--- AFTER TRAINING ---


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


 precision is: 0.8684319404220108 recall is: 0.8335980937251787 F-score is: 0.8506585612968592 accuracy is: 0.8526
